## Projet PACTE 4A-IIIA: Data Science Formula 1 World  Championship
### Sujet: L'impact de la position de depart sur le resultat d'une course.

Dans cette étude, nous nous intéressons à l’impact de la position de départ sur le résultat d’une course de Formule 1, en adoptant une approche basée sur l’analyse descriptive des données. L’objectif est de comprendre comment la position de départ influence les performances des pilotes, tout en explorant des facteurs connexes tels que les classements cumulés et les performances individuelles.

Pour cette analyse, nous utilisons plusieurs fichiers issus de la base de données Formula 1, chacun apportant des informations complémentaires :  

- **`lap_times.csv`** : Contient les temps au tour pour chaque pilote, permettant d’examiner la régularité et la performance au cours de la course.  
- **`qualifying.csv`** : Fournit les résultats des qualifications, y compris les temps enregistrés dans chaque session, et aide à relier les positions de départ aux performances en course.  
- **`races.csv`** : Offre des informations contextuelles sur chaque Grand Prix, telles que l’année, le lieu, et le circuit.  
- **`results.csv`** : Détaille les résultats spécifiques de chaque course, comme la position finale, les points marqués et les incidents de course.  
- **`status.csv`** : Indique les raisons de l’abandon ou de l’échec pour certains pilotes, utile pour analyser les événements qui perturbent les performances.  
- **`driver_standings.csv`** : Résume le classement global des pilotes après chaque course, incluant les points accumulés et les victoires totales. Ce fichier nous permet d’intégrer une perspective cumulative pour évaluer l’impact des résultats individuels sur le championnat.  

Les variables pertinentes que nous analyserons incluent :  
1. **`grid`** (fichier `results.csv`) : Position de départ sur la grille, essentielle pour comprendre son influence sur les performances en course.  
2. **`positionOrder`** (fichier `results.csv`) : Position finale à l’arrivée, pour évaluer les progrès ou les déclins des pilotes pendant la course.  
3. **`points`** (fichier `driver_standings.csv`) : Points cumulés par pilote, pour relier les performances individuelles à l’évolution globale du championnat.  
4. **`milliseconds`** (fichier `lap_times.csv`) : Temps au tour en millisecondes, pour analyser la constance des pilotes et détecter les variations de performance.  
5. **`q3`** (fichier `qualifying.csv`) : Temps réalisé lors de la troisième session de qualification, clé pour déterminer la position de départ.  
6. **`status`** (fichier `status.csv`) : Raison de l’abandon ou du résultat, pour comprendre les incidents et leur impact sur les classements.  

En analysant ces variables, nous espérons identifier des tendances et corrélations qui éclairent le rôle des qualifications, des performances en course, et des incidents dans les résultats finaux.  

In [2]:
import pandas as pd

#### 1. Chargement des données

In [3]:
lap_times = pd.read_csv("./dataset/lap_times.csv")
qualifying = pd.read_csv("./dataset/qualifying.csv") 
races = pd.read_csv("./dataset/races.csv")
results = pd.read_csv("./dataset/results.csv")
status = pd.read_csv("./dataset/status.csv")

##### Verification de chargement

In [4]:
# Vérification des datasets importés
datasets = {
    "lap_times": lap_times,
    "qualifying": qualifying,
    "races": races,
    "results": results,
    "status": status
}

for name, df in datasets.items():
    print(f"{name}: {df.shape[0]} lignes, {df.shape[1]} colonnes")

lap_times: 575029 lignes, 6 colonnes
qualifying: 10254 lignes, 9 colonnes
races: 1125 lignes, 18 colonnes
results: 26519 lignes, 18 colonnes
status: 139 lignes, 2 colonnes


#### 2. Préparation des données

#### 2.1. Exploration initiale des données
Cette étape permet d’avoir une vue d’ensemble des datasets (colonnes, types de données, valeurs manquantes).

In [5]:
# Afficher les premières lignes et résumés des datasets
for name, df in datasets.items():
    print(f"Dataset: {name}")
    print(df.info())
    print(df.head(), "\n")


Dataset: lap_times
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575029 entries, 0 to 575028
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   raceId        575029 non-null  int64 
 1   driverId      575029 non-null  int64 
 2   lap           575029 non-null  int64 
 3   position      575029 non-null  int64 
 4   time          575029 non-null  object
 5   milliseconds  575029 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 26.3+ MB
None
   raceId  driverId  lap  position      time  milliseconds
0     841        20    1         1  1:38.109         98109
1     841        20    2         1  1:33.006         93006
2     841        20    3         1  1:32.713         92713
3     841        20    4         1  1:32.803         92803
4     841        20    5         1  1:32.342         92342 

Dataset: qualifying
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10254 entries, 0 to 10253
Data columns (total

##### 2.2. Traitement des valeurs manquantes
Les valeurs manquantes peuvent fausser les résultats. On identifie ces valeurs et choisit une stratégie : suppression, imputation, ou autre.

In [13]:
# Vérification des valeurs manquantes
for name, df in datasets.items():
    print(f"{name} - Valeurs manquantes:\n{df.isnull().sum()}\n")

circuits - Valeurs manquantes:
circuitId     0
circuitRef    0
name          0
location      0
country       0
lat           0
lng           0
alt           0
url           0
dtype: int64

constructor_results - Valeurs manquantes:
constructorResultsId    0
raceId                  0
constructorId           0
points                  0
status                  0
dtype: int64

constructor_standings - Valeurs manquantes:
constructorStandingsId    0
raceId                    0
constructorId             0
points                    0
position                  0
positionText              0
wins                      0
dtype: int64

constructors - Valeurs manquantes:
constructorId     0
constructorRef    0
name              0
nationality       0
url               0
dtype: int64

driver_standings - Valeurs manquantes:
driverStandingsId    0
raceId               0
driverId             0
points               0
position             0
positionText         0
wins                 0
dtype: int64

drivers 

#### 3. Analyse descriptive des données

##### 3.1. Statistiques descriptives
Fournir des statistiques (moyenne, médiane, écart-type, etc.) pour comprendre la distribution des variables.